In [1]:
# Install Pytorch & other libraries
%pip install -q tensorboard wandb 
 
# Install Hugging Face libraries
%pip install -q --upgrade \
  "transformers==4.45.1" \
  "datasets==3.0.1" \
  "accelerate==0.34.2" \
  "evaluate==0.4.3" \
  "bitsandbytes==0.44.0" \
  "trl==0.11.1" \
  "peft==0.13.0" \
  "qwen-vl-utils"


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install pillow==9.4.0


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [13]:
prompt= """질문: {korean_question}
선택지: {korean_choices}
힌트: {korean_hint}"""

system_message = "주어진 이미지와 질문을 바탕으로 답변하세요.\n이때 정답은 선택지 중 1개를 선택해야하며 힌트가 주어질 수 있습니다. 가장 적절한 답을 1개 선택하세요."

In [14]:
from datasets import load_dataset

# 데이터셋을 OpenAI 메시지 형식으로 변환하는 함수      
def format_data(sample):
   return {"messages": [
               {
                   # 시스템 프롬프트
                   "role": "system", 
                   "content": [{"type": "text", "text": system_message}], 
               },
               {
                   # 유저 프롬프트
                   "role": "user",  
                   "content": [
                       {
                           "type": "text",
                           "text": prompt.format(
                              korean_question=sample["korean_question"], 
                              korean_choices=sample["korean_choices"], 
                              korean_hint=sample["korean_hint"]
                              ),
                       },{
                           "type": "image", 
                           "image": sample["image"] 
                                if sample["image"] is not None else "", 
                       }
                   ],
               },
               {
                   # AI 어시스턴트 답변
                   
                   "role": "assistant", 
                   "content": [
                        {
                            "type": "text", 
                            "text": sample["answer_str"]
                        }
                    ], 
               },
           ],
       }

# 허브에서 데이터셋 로드 및 이미지가 존재하는 경우만 필터링
dataset = load_dataset("daje/Ko-SciecneQA", split="train")
dataset = dataset.filter(lambda example: example["image"] is not None)
dataset = [format_data(sample) for sample in dataset]

In [15]:
test_dataset = dataset[int(len(dataset) * 0.9):]

In [17]:
print(test_dataset[0])

{'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': '주어진 이미지와 질문을 바탕으로 답변하세요.\n이때 정답은 선택지 중 1개를 선택해야하며 힌트가 주어질 수 있습니다. 가장 적절한 답을 1개 선택하세요.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': "질문: 휘트니의 실험이 가장 잘 답할 수 있는 질문을 선택하세요.\n선택지: ['계란이 민물에서 더 잘 뜨나요, 아니면 소금물에서 더 잘 뜨나요?', '유리잔에 있는 물의 양이 계란이 물에 가라앉거나 뜨는 것에 영향을 미치나요?']\n힌트: 아래의 글은 실험을 설명합니다. 글을 읽고 아래의 지시를 따르세요.\n\n휘트니는 여섯 개의 유리잔 각각에 4온스의 물을 부었습니다. 휘트니는 세 개의 유리잔에 각각 한 스푼의 소금을 녹였고, 나머지 세 개에는 소금을 넣지 않았습니다. 그런 다음, 휘트니는 한 유리잔에 계란을 넣고 계란이 뜨는지 관찰했습니다. 그녀는 계란을 꺼내어 말렸습니다. 그녀는 다른 다섯 개의 유리잔에서도 이 과정을 반복하며 계란이 뜨는지 기록했습니다. 휘트니는 이 테스트를 두 개의 계란으로 더 반복하여 민물과 소금물에서 계란이 뜨는 횟수를 비교했습니다.\n그림: 소금물에 떠 있는 계란."}, {'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=302x232 at 0x7EE6C4016470>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '계란이 민물에서 더 잘 뜨나요, 아니면 소금물에서 더 잘 뜨나요?'}]}]}


In [8]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor
 
# 허깅페이스에서 제공하는 Qwen 시리즈의 비전-언어 모델 ID
model_id = "Qwen/Qwen2-VL-7B-Instruct"
 
# 모델과 프로세서 로드
model = AutoModelForVision2Seq.from_pretrained(
   model_id,
   # 가용한 GPU 메모리에 모델을 자동으로 최적 할당하는 설정
   device_map="auto",     
   # 메모리 효율과 연산 속도를 위해 bfloat16 형식의 부동소수점 정밀도 사용             
   torch_dtype=torch.bfloat16,                  
)
 
# 입력 텍스트와 이미지를 모델이 이해할 수 있는 형태로 변환하는 전처리기 로드
processor = AutoProcessor.from_pretrained(model_id)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model-00002-of-00005.safetensors:  26%|##5       | 996M/3.86G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [9]:
# 모델 답변을 생성하는 함수
def generate_description(messages, model, processor):
   # 추론을 위한 준비
   text = processor.apply_chat_template(
       messages, tokenize=False, add_generation_prompt=True
   )
   image_inputs, video_inputs = process_vision_info(messages)
   inputs = processor(
       text=[text],
       images=image_inputs,
       videos=video_inputs,
       padding=True,
       return_tensors="pt",
   )
    
   inputs = inputs.to(model.device)
   # 추론: 출력 생성
   generated_ids = model.generate(
      **inputs, 
      max_new_tokens=128,
      top_p=1.0, 
      do_sample=True, 
      temperature=0.1
      )
   generated_ids_trimmed = [
      out_ids[len(in_ids) :] 
      for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]

    # 출력 부분만
   output_text = processor.batch_decode(
       generated_ids_trimmed, 
       skip_special_tokens=True, 
       clean_up_tokenization_spaces=False
   )
   return output_text[0]

In [20]:
from qwen_vl_utils import process_vision_info

# 테스트 데이터 1개 출력
messages =  test_dataset[0]["messages"]
print(messages)

[{'role': 'system', 'content': [{'type': 'text', 'text': '주어진 이미지와 질문을 바탕으로 답변하세요.\n이때 정답은 선택지 중 1개를 선택해야하며 힌트가 주어질 수 있습니다. 가장 적절한 답을 1개 선택하세요.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': "질문: 휘트니의 실험이 가장 잘 답할 수 있는 질문을 선택하세요.\n선택지: ['계란이 민물에서 더 잘 뜨나요, 아니면 소금물에서 더 잘 뜨나요?', '유리잔에 있는 물의 양이 계란이 물에 가라앉거나 뜨는 것에 영향을 미치나요?']\n힌트: 아래의 글은 실험을 설명합니다. 글을 읽고 아래의 지시를 따르세요.\n\n휘트니는 여섯 개의 유리잔 각각에 4온스의 물을 부었습니다. 휘트니는 세 개의 유리잔에 각각 한 스푼의 소금을 녹였고, 나머지 세 개에는 소금을 넣지 않았습니다. 그런 다음, 휘트니는 한 유리잔에 계란을 넣고 계란이 뜨는지 관찰했습니다. 그녀는 계란을 꺼내어 말렸습니다. 그녀는 다른 다섯 개의 유리잔에서도 이 과정을 반복하며 계란이 뜨는지 기록했습니다. 휘트니는 이 테스트를 두 개의 계란으로 더 반복하여 민물과 소금물에서 계란이 뜨는 횟수를 비교했습니다.\n그림: 소금물에 떠 있는 계란."}, {'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=302x232 at 0x7EE6C4016470>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '계란이 민물에서 더 잘 뜨나요, 아니면 소금물에서 더 잘 뜨나요?'}]}]


In [21]:
base_description = generate_description(messages, model, processor)
print(base_description)

'유리잔에 있는 물의 양이 계란이 물에 가라앉거나 뜰하는 것에 영향을 미치나요?'


In [22]:
# 테스트 데이터 전체에 대해서 출력
from tqdm.auto import tqdm 

no_train_result = [] 

for idx in tqdm(range(len(test_dataset))):
    messages =  test_dataset[idx]["messages"]
    base_description = generate_description(messages, model, processor)
    no_train_result.append(base_description)

  0%|          | 0/622 [00:00<?, ?it/s]

In [23]:
# 정답 추출
answers = [data["messages"][2]["content"][0]["text"] for data in test_dataset]
answers[:2]

['계란이 민물에서 더 잘 뜨나요, 아니면 소금물에서 더 잘 뜨나요?', '두 쌍의 자기력 크기는 같습니다.']

In [26]:
# 정확도 계산
hit = 0
for inference, answer in zip(no_train_result, answers):
    if inference == answer : 
        hit += 1
print('맞춘 샘플 개수:',hit)
print('정확도:', hit/len(answers) * 100)

맞춘 샘플 개수: 448
정확도: 72.02572347266882


In [30]:
answers[:10]

['계란이 민물에서 더 잘 뜨나요, 아니면 소금물에서 더 잘 뜨나요?',
 '두 쌍의 자기력 크기는 같습니다.',
 '매사추세츠',
 '피닉스',
 '용액 B',
 '델라웨어',
 '샘플 B',
 '켄터키',
 '호주',
 '밀어내다']

In [31]:
no_train_result[:10]

["'유리잔에 있는 물의 양이 계란이 물에 가라앉거나 뜰하는 것에 영향을 미치나요?'",
 '두 쌍의 자기력 크기는 같습니다.',
 '매사추세츠',
 '피닉스',
 '용액 B',
 '메릴랜드',
 '샘플 B',
 '메인',
 '호주',
 '밀어내다']

In [34]:
!rm -rf .cache/huggingface/hub/models--Qwen/Qwen2-VL-7B-Instruct/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
# 허깅페이스에서 제공하는 Qwen 시리즈의 비전-언어 모델 ID
lora_model_id = "iamjoon/Qwen2-VL-7B-instruct-KoScienceQA"
 
# 모델과 프로세서 로드
lora_model = AutoModelForVision2Seq.from_pretrained(
   lora_model_id,
   # 가용한 GPU 메모리에 모델을 자동으로 최적 할당하는 설정
   device_map="auto",     
   # 메모리 효율과 연산 속도를 위해 bfloat16 형식의 부동소수점 정밀도 사용             
   torch_dtype=torch.bfloat16,                  
)

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00018.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00002-of-00018.safetensors:   0%|          | 0.00/2.18G [00:00<?, ?B/s]

...

model-00016-of-00018.safetensors:   0%|          | 0.00/1.86G [00:00<?, ?B/s]

model-00017-of-00018.safetensors:   0%|          | 0.00/2.18G [00:00<?, ?B/s]

model-00018-of-00018.safetensors:   0%|          | 0.00/815M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/18 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [35]:
# 학습한 모델로 예측하기 
results = [] 
for idx in tqdm(range(len(test_dataset))):
    messages =  test_dataset[idx]["messages"]
    ft_description = generate_description(messages, lora_model, processor)
    results.append(ft_description)

  0%|          | 0/622 [00:00<?, ?it/s]

In [36]:
# 정확도 계산
hit = 0
for inference, answer in zip(results, answers):
    if inference == answer : 
        hit += 1
print('맞춘 샘플 개수:',hit)
print('정확도:', hit/len(answers) * 100)

맞춘 샘플 개수: 609
정확도: 97.90996784565917
